## Data Manipulation Tutorial

This notebook provides examples on how to carry out data manipulation and aggregation using the post_processing python library. Be sure to go through the [Quick Start](https://nhs-postprocessing.readthedocs.io/en/stable/QuickStart.html) section of the [documentation](https://nhs-postprocessing.readthedocs.io/en/stable/index.html) for instructions on how to access and import the libary and its packages.

If you would like to open an editable runnable version of the tutorial click [here](https://mybinder.org/v2/gh/UchechukwuUdenze/NHS_PostProcessing/main?%2FHEAD=&urlpath=%2Fdoc%2Ftree%2Fdocs%2Fsource%2Fnotebooks%2Ftutorial-data-manipulation.ipynb) to be directed to a binder platform

<mark>The Library is still under active development and empty sections will be completed in Due time</mark>

### Table of content
- [Generate DataFrames](#generate-dataframes)
- [Daily Aggregation](#daily-aggregation)
- [Weekly Aggregation](#weekly-aggregation)
- [Monthly Aggregation](#monthly-aggregation)
- [Yearly Aggregation](#yearly-aggregation)
- [Statistics Aggregation](#statistics-aggregation)
- [Periodic/Seasaonal Aggregation](#periodic/seasonal-aggregation)
- [Long Term Aggregation](#long-term-aggregation)

 All files are available in the github repository [here](https://github.com/UchechukwuUdenze/NHS_PostProcessing/tree/main/docs/source/notebooks)

### Requirements

The conda environmnent contains all libraries associated the post processing library. After setting up the conda environment, you only have to import the data maniupulation module from postprocessinglib.evaluation.

In this example though, I will also be importing other modules to help generate the data that I will be trying to analyse.

In [1]:
### Remove and modify these later.
import sys
import pandas as pd
sys.path.append("../../../")

In [2]:
from postprocessinglib.evaluation import data

### GENERATE DATAFRAMES

This is the main overarching function that returns the required files in theier respective formats for use by the other modules and functions in the library. In its simplest form, it requires a csv file which contains the predicted and measured data formatted reffered to as the **merged data** as shown below :

 **merged data**
 
| Some datetime  | station1_obs | station1_sim | station2_obs | station2_sim |
| -------------- | ------------ | ------------ | ------------ | ------------ | 

or two csv files - one for observed and one for the simulated data, similarly formatted as shown below:

 **obs data**
 
| Some datetime  | station1_obs | station2_obs |
| -------------- | ------------ | ------------ |
 
 **sim data**

| Some datetime  | station1_sim | station2_sim |
| -------------- | ------------ | ------------ |


We then pass these into our generate dataframes function as shown below:

In [3]:
# passing a small controlled csv file with only two stations for testing
path = "MESH_output_streamflow_1.csv"

# csv_fpath is used to represent the merged csv file
DATAFRAMES = data.generate_dataframes(csv_fpaths=path) 

The start date for the Data is 1980-01-01


By default the function returns a dictionary that contains 3 dataframes - the Merged dataframe, the Observed dataframe and the Simuated dataframe, represented as DF, DF_OBSERVED and DF_SIMULATED respectively as demonstrated below:

In [4]:
print("The Merged dataframe:")
print(DATAFRAMES["DF"].head(5))
print("\nThe Observed dataframe:")
print(DATAFRAMES["DF_OBSERVED"].head(5))
print("\nThe Simulated dataframe:")
print(DATAFRAMES["DF_SIMULATED"].head(5))

The Merged dataframe:
            QOMEAS_05BB001  QOSIM_05BB001  QOMEAS_05BA001  QOSIM_05BA001
1980-01-01            15.0     940.934800             NaN     695.375700
1980-01-02            15.0    1000.471000             NaN     233.646000
1980-01-03            13.3     303.156100             NaN       5.282405
1980-01-04            11.8      13.658420             NaN       0.481292
1980-01-05            12.9       2.095571             NaN       0.298271

The Observed dataframe:
            QOMEAS_05BB001  QOMEAS_05BA001
1980-01-01            15.0             NaN
1980-01-02            15.0             NaN
1980-01-03            13.3             NaN
1980-01-04            11.8             NaN
1980-01-05            12.9             NaN

The Simulated dataframe:
            QOSIM_05BB001  QOSIM_05BA001
1980-01-01     940.934800     695.375700
1980-01-02    1000.471000     233.646000
1980-01-03     303.156100       5.282405
1980-01-04      13.658420       0.481292
1980-01-05       2.095571 

You are also able to tell the function to skip the first few values by writing a value to the **warm_up** parameter of the function. We are also able to specify a start and end date using the **start_date**  and **end_date**. These are useful in cases when you want only a fixed time like a particular year everything after or before a particular date. A few examples are shown below:

In [5]:
# assuming the simulation model needs 366 days (the first year) to warm up and account for errors during the learning phase.
DATAFRAMES = data.generate_dataframes(csv_fpaths=path, warm_up=366)

The start date for the Data is 1981-01-01


Observe that the data now skips the entire first year starting from 1981-01-01 as opposed to 1980-01-01.

In [6]:
DATAFRAMES_till2009 = data.generate_dataframes(csv_fpaths=path, warm_up=366, end_date='2009-12-31')
print("\nThe End of the Merged dataframe:")
print(DATAFRAMES_till2009["DF"].tail(5))

The start date for the Data is 1981-01-01

The End of the Merged dataframe:
            QOMEAS_05BB001  QOSIM_05BB001  QOMEAS_05BA001  QOSIM_05BA001
2009-12-27             NaN       4.114114             NaN       0.815359
2009-12-28             NaN       4.091105             NaN       0.810912
2009-12-29             NaN       4.068261             NaN       0.806497
2009-12-30             NaN       4.045577             NaN       0.802113
2009-12-31             NaN       4.023057             NaN       0.797758


Notice how it ends at 2009 as specified

In [7]:
DATAFRAMES_from1995 = data.generate_dataframes(csv_fpaths=path, warm_up=366, start_date='1995-01-01')
print("\nThe Start of the Observed dataframe:")
print(DATAFRAMES_from1995["DF_OBSERVED"].head(5))

The start date for the Data is 1995-01-01

The Start of the Observed dataframe:
            QOMEAS_05BB001  QOMEAS_05BA001
1995-01-01            8.37             NaN
1995-01-02           10.10             NaN
1995-01-03           12.20             NaN
1995-01-04           13.00             NaN
1995-01-05           13.20             NaN


Observe that the data now starts 1995

In [8]:
DATAFRAMES_January2010 = data.generate_dataframes(csv_fpaths=path, warm_up=366, start_date='2010-01-01' , end_date='2010-1-31')
print("\nThe Start of the Merged dataframe:")
print(DATAFRAMES_January2010["DF"].head(5))
print("\nThe End of the Merged dataframe:")
print(DATAFRAMES_January2010["DF"].tail(5))

The start date for the Data is 2010-01-01

The Start of the Merged dataframe:
            QOMEAS_05BB001  QOSIM_05BB001  QOMEAS_05BA001  QOSIM_05BA001
2010-01-01             NaN       4.000698             NaN       0.793435
2010-01-02             NaN       3.978494             NaN       0.789141
2010-01-03             NaN       3.956450             NaN       0.784877
2010-01-04             NaN       3.934558             NaN       0.780643
2010-01-05             NaN       3.912824             NaN       0.776437

The End of the Merged dataframe:
            QOMEAS_05BB001  QOSIM_05BB001  QOMEAS_05BA001  QOSIM_05BA001
2010-01-27             NaN       3.471175             NaN       0.690854
2010-01-28             NaN       3.452648             NaN       0.687258
2010-01-29             NaN       3.434252             NaN       0.683687
2010-01-30             NaN       3.415977             NaN       0.680139
2010-01-31             NaN       3.397829             NaN       0.676615


Observe that the data now starts from January 1st 2010 as specified and ends at January 31st 2010 

Something to note though is that when specifying a warm up date and a start date and the start date exists before the warm up, its start date will be pushed forward to the warm up date as the warm up parameter takes precedence. For instance:

In [9]:
# Here we set our start date to be somewhere within the 366 warmup time. It will get overidden and start from the end of
# the warm up time
DATAFRAMES_from_June_1980 = data.generate_dataframes(csv_fpaths=path, warm_up=366, start_date='1980-06-01')
print("\nThe Start of the Predicted Data: ")
print(DATAFRAMES_from_June_1980["DF_SIMULATED"].head(5))

The start date for the Data is 1981-01-01

The Start of the Predicted Data: 
            QOSIM_05BB001  QOSIM_05BA001
1981-01-01       2.518999       1.001954
1981-01-02       2.507289       0.997078
1981-01-03       2.495637       0.992233
1981-01-04       2.484073       0.987417
1981-01-05       2.472571       0.982631


As you can observe it starts from 1981 despite specifying a start date of June 1st 1980!

The three dataframes - merged, observed and simulated - form the backbone of the library. Every other function in the library uses one or more of at least these three dataframes to perform analysis whether visual, descriptive or diagonistic. 

### DAILY AGGREGATION
This function returns the daily aggregate of the data passed into it. Its aggregates using the method passed or if one isnt given, its default is **mean**. Its that simple. Most of the data already comes with daily time stamps though so this is one of the fewer used functions. Its functionality is shown below: 

In [10]:
data.daily_aggregate(df=DATAFRAMES["DF_MERGED"])

Station1           Station2          
           QOMEAS    QOSIM1   QOMEAS    QOSIM1
1981/001     9.85  2.518999      NaN  1.001954
1981/002    10.20  2.507289      NaN  0.997078
1981/003    10.00  2.495637      NaN  0.992233
1981/004    10.10  2.484073      NaN  0.987417
1981/005     9.99  2.472571      NaN  0.982631
...           ...       ...      ...       ...
2017/361      NaN  4.418050      NaN  1.380227
2017/362      NaN  4.393084      NaN  1.372171
2017/363      NaN  4.368303      NaN  1.364174
2017/364      NaN  4.343699      NaN  1.356237
2017/365      NaN  4.319275      NaN  1.348359

[13514 rows x 4 columns]

It returns a dataframe indexed daily from 1 till 365/366 i.e., the days of the year

### WEEKLY AGGREGATION
This function returns the weekly aggregate of the data passed into it. Its aggregates using the method passed or if one isnt given, its default is **mean**. Its that simple. Its functionality is shown below:

In [11]:
data.weekly_aggregate(df=DATAFRAMES["DF_MERGED"]) # default method of aggregation is mean

Station1           Station2          
               QOMEAS    QOSIM1   QOMEAS    QOSIM1
1980-12-29  10.037500  2.501499      NaN  0.994671
1981-01-05   9.244286  2.438589      NaN  0.968506
1981-01-12   8.461429  2.361289      NaN  0.936405
1981-01-19   8.345714  2.287077      NaN  0.905643
1981-01-26   8.461429  2.215803      NaN  0.876150
...               ...       ...      ...       ...
2017-11-27        NaN  5.165260      NaN  1.621172
2017-12-04        NaN  4.958076      NaN  1.554878
2017-12-11        NaN  4.760181      NaN  1.490809
2017-12-18        NaN  4.572103      NaN  1.429983
2017-12-25        NaN  4.393448      NaN  1.372291

[1931 rows x 4 columns]

It returns a dataframe indexed weekly from 0/1 till 52/53 i.e., the weeks of the year

In [12]:
data.weekly_aggregate(df=DATAFRAMES["DF_MERGED"], method='sum') # here we aggregate by summing up all the values of the week.

Station1            Station2           
             QOMEAS     QOSIM1   QOMEAS     QOSIM1
1980-12-29    40.15  10.005998      0.0   3.978683
1981-01-05    64.71  17.070122      0.0   6.779542
1981-01-12    59.23  16.529020      0.0   6.554838
1981-01-19    58.42  16.009540      0.0   6.339498
1981-01-26    59.23  15.510619      0.0   6.133052
...             ...        ...      ...        ...
2017-11-27     0.00  36.156822      0.0  11.348205
2017-12-04     0.00  34.706534      0.0  10.884148
2017-12-11     0.00  33.321266      0.0  10.435665
2017-12-18     0.00  32.004719      0.0  10.009884
2017-12-25     0.00  30.754137      0.0   9.606034

[1931 rows x 4 columns]

### YEARLY AGGREGATION
This function returns the yearly aggregate of the data passed into it. Its aggregates using the method passed or if one isnt given, its default is **mean**. Its that simple. Its functionality is shown below:

In [13]:
data.yearly_aggregate(df=DATAFRAMES["DF_MERGED"]) # default method of aggregation is mean

Station1              Station2           
            QOMEAS     QOSIM1     QOMEAS     QOSIM1
1981-01   8.815806   2.352880        NaN   0.932961
1981-02   7.780000   2.060198        NaN   0.811975
1981-03   6.896129   1.812920        NaN   0.710498
1981-04   7.981333   3.132911        NaN   0.821663
1981-05  43.538710  50.736276  10.111333  15.202072
...            ...        ...        ...        ...
2017-08        NaN  32.222317        NaN  17.704763
2017-09        NaN  28.141430        NaN  14.315134
2017-10        NaN   7.698483        NaN   2.615914
2017-11        NaN   5.625516        NaN   1.770196
2017-12        NaN   4.712923        NaN   1.475527

[444 rows x 4 columns]

It returns a dataframe indexed yearly from the first year in your data till the last year. 

In [14]:
data.yearly_aggregate(df=DATAFRAMES["DF_MERGED"], method='median') # here we aggregate by finding the median of the values each year.

Station1            Station2           
          QOMEAS     QOSIM1   QOMEAS     QOSIM1
1981-01    8.590   2.350375      NaN   0.931876
1981-02    7.825   2.058540      NaN   0.811263
1981-03    6.860   1.811243      NaN   0.709789
1981-04    7.400   1.720303      NaN   0.634707
1981-05   15.800   5.115736    5.165   1.589602
...          ...        ...      ...        ...
2017-08      NaN  31.761380      NaN  17.606670
2017-09      NaN  25.176370      NaN  12.297040
2017-10      NaN   6.744768      NaN   2.161909
2017-11      NaN   5.622982      NaN   1.767848
2017-12      NaN   4.705044      NaN   1.472965

[444 rows x 4 columns]

### MONTHLY AGGREGATION
This function returns the monthly aggregate of the data passed into it. Its aggregates using the method passed or if one isnt given, its default is **mean**. Its that simple. Its functionality is shown below:

In [15]:
data.monthly_aggregate(df=DATAFRAMES["DF_MERGED"]) # default method of aggregation is mean

Station1              Station2           
            QOMEAS     QOSIM1     QOMEAS     QOSIM1
1981-01   8.815806   2.352880        NaN   0.932961
1981-02   7.780000   2.060198        NaN   0.811975
1981-03   6.896129   1.812920        NaN   0.710498
1981-04   7.981333   3.132911        NaN   0.821663
1981-05  43.538710  50.736276  10.111333  15.202072
...            ...        ...        ...        ...
2017-08        NaN  32.222317        NaN  17.704763
2017-09        NaN  28.141430        NaN  14.315134
2017-10        NaN   7.698483        NaN   2.615914
2017-11        NaN   5.625516        NaN   1.770196
2017-12        NaN   4.712923        NaN   1.475527

[444 rows x 4 columns]

It returns a dataframe indexed monthly from 1 till 12 i.e., the months of the year

In [16]:
data.monthly_aggregate(df=DATAFRAMES["DF_MERGED"], method='max') # here we aggregate by returning the maximum value that month

Station1             Station2           
          QOMEAS      QOSIM1   QOMEAS     QOSIM1
1981-01    10.20    2.518999      NaN   1.001954
1981-02     8.51    2.186006      NaN   0.863836
1981-03     7.50    1.931955      NaN   0.759228
1981-04    15.30   17.241560      NaN   3.692734
1981-05   165.00  220.485800     36.6  97.054240
...          ...         ...      ...        ...
2017-08      NaN   41.657390      NaN  21.751910
2017-09      NaN  116.866500      NaN  53.925540
2017-10      NaN   19.746100      NaN   8.157081
2017-11      NaN    6.091304      NaN   1.927400
2017-12      NaN    5.134585      NaN   1.611400

[444 rows x 4 columns]

### STATISTICS AGGREGATION

This allows us to calculate the aggregate of all the simulations accross the dataframe for every datetime index. Its aggregates using the method passed or if one isnt given, its default is **mean**. Unlike the other methods of aggregation, we are also able to perform quantile calculations with this method of aggregation. Its that simple. Its functionality is shown below:

In [17]:
data.stat_aggregate(df=DATAFRAMES["DF_MERGED"], method='max')

,Station1,Station2
,MAX,MAX
1981-01-01,2.518999,1.001954
1981-01-02,2.507289,0.997078
1981-01-03,2.495637,0.992233
1981-01-04,2.484073,0.987417
1981-01-05,2.472571,0.982631
...,...,...
2017-12-27,4.418050,1.380227
2017-12-28,4.393084,1.372171
2017-12-29,4.368303,1.364174


In [18]:
data.stat_aggregate(df=DATAFRAMES["DF_MERGED"], method='q75')

,Station1,Station2
,Q75,Q75
1981-01-01,2.518999,1.001954
1981-01-02,2.507289,0.997078
1981-01-03,2.495637,0.992233
1981-01-04,2.484073,0.987417
1981-01-05,2.472571,0.982631
...,...,...
2017-12-27,4.418050,1.380227
2017-12-28,4.393084,1.372171
2017-12-29,4.368303,1.364174


### PERIODIC/SEASONAL AGGREGATION
This allows us to return a specific period of time for every year or a select few years within a data set. Its allows you to essentially analyse a **season** or **period** every year without having to look through every day. 
For exmaple, lets say we want to isolate what the streamflow was like on the first 2 days of January every year, we would go..

In [19]:
data.seasonal_period(df=DATAFRAMES["DF"], daily_period=('01-01', '01-02'))

,QOMEAS_05BB001,QOSIM_05BB001,QOMEAS_05BA001,QOSIM_05BA001
1981-01-01,9.85,2.518999,NaN,1.001954
1981-01-02,10.20,2.507289,NaN,0.997078
1982-01-01,7.17,5.465301,NaN,2.429704
1982-01-02,7.02,5.433753,NaN,2.414755
1983-01-01,8.98,5.371416,NaN,2.441398
...,...,...,...,...
2015-01-02,NaN,6.944578,NaN,1.615503
2016-01-01,NaN,3.686424,NaN,1.005240
2016-01-02,NaN,3.666536,NaN,0.999701
2017-01-01,NaN,2.700768,NaN,0.872387


Observe that for every year, we only get the first two days of the year. 
We can also specify specific years and not return every year. For example we can get the predicted values for the first week of summer for the years 1999, 2001 and 2005 as shown below:

In [20]:
data.seasonal_period(df=DATAFRAMES["DF_SIMULATED"], daily_period=('06-21', '06-28'), years=[1999, 2001, 2005])

,QOSIM_05BB001,QOSIM_05BA001
1999-06-21,155.64250,127.26980
1999-06-22,147.19820,74.31406
1999-06-23,80.06104,34.51432
1999-06-24,45.07025,38.66659
1999-06-25,129.59160,104.75280
1999-06-26,156.33070,51.86972
1999-06-27,166.94590,62.61042
1999-06-28,85.17047,27.95974
2001-06-21,42.08820,32.47492
2001-06-22,44.51981,29.40924


As you can see, we are able to get the first week of summer for those 3 years.

### LONG TERM AGGREGATION

this allows us to compute the long-term seasonal aggregate values of a given DataFrame by applying the specified aggregation method to each day across all years in the provided time period. The resulting data is aggregated into a single year (1 to 365/366 days). This way we are able to see how the models perform year in year out compared to the actual recorded data - both aggregated as necessary. An example is shown below:

In [21]:
data.long_term_seasonal(df=DATAFRAMES["DF"]) # As usual the default aggregation method is mean/average

,QOMEAS_05BB001,QOSIM_05BB001,QOMEAS_05BA001,QOSIM_05BA001
jday,,,,
1,9.446471,4.037666,NaN,1.130686
2,9.428125,4.014474,NaN,1.123915
3,9.660625,3.991451,NaN,1.117196
4,9.804375,3.968602,NaN,1.110529
5,9.787500,3.945921,NaN,1.103913
...,...,...,...,...
362,9.942500,4.188140,NaN,1.169614
363,9.695000,4.163847,NaN,1.162533
364,9.633125,4.139735,NaN,1.155507


We are also able to calculate quantiles, For example the 75th Quantile value for all years aggregated into a single year looks like:

In [22]:
data.long_term_seasonal(df=DATAFRAMES["DF_SIMULATED"], method = 'Q75')

,QOSIM_05BB001,QOSIM_05BA001
jday,,
1,4.830453,1.315370
2,4.801530,1.306986
3,4.772831,1.298670
4,4.744344,1.290422
5,4.716085,1.282241
...,...,...
362,4.978491,1.372171
363,4.948421,1.364174
364,4.918590,1.356237


### **Note**
All of these functions with their various means of aggregation are available as individual functions but they can also be generated right from the **generate_dataframes()** function if you know eaxactly what you'll need from the beginning. It just requires specifying a few more parameters. These parameters are shown below: 

In [23]:
## Lets use a time period of 1981 to 1990 to demonstrate this
DATAFRAMES = data.generate_dataframes(csv_fpaths=path, warm_up=365, start_date = "1981-01-01", end_date = "1990-12-31",
                                      # optional arguments                                      
                                      # you specify that you want an aggregated dataframe by passing 'True' into 
                                      # the respective parameter and then you pass in your preffered method of aggregation
                                      
                                      # If you want daily aggregation
                                      daily_agg = True, da_method = 'min',
                                      # lets see a weekly aggregation
                                      weekly_agg = True, wa_method = 'min', # we want the minimum value each week
                                      # lets also see monthly aggregation
                                      monthly_agg = True, ma_method = 'inst', # we want the maximum value each month
                                      # lets also see yearly aggregation
                                      yearly_agg = True, ya_method = 'sum', # we want the sum of all values each year
                                      # lets see the stats aggregation
                                      stat_agg = True, stat_method = 'q75'
                                      # note that without inputing the respective methods,
                                      # the functions will still default to mean as the method of aggregation
                                     )

The start date for the Data is 1981-01-01


In [24]:
DATAFRAMES = data.generate_dataframes(csv_fpaths=path, warm_up=365, start_date = "1981-01-01", end_date = "1990-12-31",
                                      # seasonal aggregation
                                      # obtaining the months of May till August from every year from 1981 to 1985
                                      seasonal_p = True, sp_dperiod = ('05-01', '08-30'),
                                      sp_subset = ('1981-01-01', '1985-12-31'),
                                      # instead of sp_subset, we can also use years = [1981, 1982, 1983, 1984, 1985].
                                      
                                      # long term seasonal aggregation
                                      long_term = True, lt_method = ["q33.33", "median" ,'q75' ,'Q25' ,'q33' ],
                                      # when using long term in the generate_dataframes function, we are able to pass
                                      # in a list of methods of aggregation we want generated. BY dafault though it will
                                      # always generate maximum, minimum and median value dataframes 
                                     )

The start date for the Data is 1981-01-01


Putting it all together, we have:

In [25]:
DATAFRAMES = data.generate_dataframes(csv_fpaths=path, warm_up=365, start_date = "1981-01-01", end_date = "1990-12-31",
                                      daily_agg = True, da_method = 'min',
                                      weekly_agg = True, wa_method = 'min',
                                      monthly_agg = True, ma_method = 'inst', 
                                      yearly_agg = True, ya_method = 'sum',
                                      stat_agg = True, stat_method = 'q75',
                                      seasonal_p = True, sp_dperiod = ('05-01', '08-30'), sp_subset = ('1981-01-01', '1985-12-31'),
                                      long_term = True, lt_method = ["q33.33", "median" ,'q75' ,'Q25' ,'q33' ],
                                     )


for key, value in DATAFRAMES.items():
    print(f"{key}:\n{value}")

The start date for the Data is 1981-01-01
DF:
            QOMEAS_05BB001  QOSIM_05BB001  QOMEAS_05BA001  QOSIM_05BA001
1981-01-01            9.85       2.518999             NaN       1.001954
1981-01-02           10.20       2.507289             NaN       0.997078
1981-01-03           10.00       2.495637             NaN       0.992233
1981-01-04           10.10       2.484073             NaN       0.987417
1981-01-05            9.99       2.472571             NaN       0.982631
...                    ...            ...             ...            ...
1990-12-27           10.10       6.615961             NaN       1.737144
1990-12-28            9.50       6.573054             NaN       1.725025
1990-12-29            8.60       6.530500             NaN       1.713013
1990-12-30            8.20       6.488300             NaN       1.701107
1990-12-31            8.25       6.446449             NaN       1.689308

[3652 rows x 4 columns]
DF_OBSERVED:
            QOMEAS_05BB001  QOMEAS_05BA0